In [217]:
import pandas as pd
import numpy as np
import chardet

In [218]:
def smart_read_csv(file_path) -> pd.DataFrame:
    with open(file_path,'rb') as f:
        res = chardet.detect(f.read(1000))

    return pd.read_csv(file_path, encoding=res['encoding'], encoding_errors='replace')

try:

    raw_orders = smart_read_csv('../data/orders.csv')
    raw_orders_details = smart_read_csv('../data/order_details.csv')
    raw_pizzas = smart_read_csv('../data/pizzas.csv')
    raw_pizza_types = smart_read_csv('../data/pizza_types.csv')

    print(f"Lectura exitosa de los archivos csv.")
except Exception as e:
    print(f"Error en la lectura de alguno de los archivos csv: {e}")

Lectura exitosa de los archivos csv.


In [245]:
dimDate = pd.to_datetime(raw_orders['date'],errors='coerce').dt.strftime('%Y%m%d').astype(int).to_frame('date_id')
dimDate = dimDate.drop_duplicates()
dimDate['date'] = pd.to_datetime(raw_orders['date'], errors='coerce')
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.strftime('%m')
dimDate['day'] = dimDate['date'].dt.strftime('%d')
dimDate['month_name_eng'] = dimDate['date'].dt.month_name()
dimDate['month_name_spa'] = dimDate['date'].dt.month_name(locale='es_ES')
dimDate['day_name_eng'] = dimDate['date'].dt.day_name()
dimDate['day_name_spa'] = dimDate['date'].dt.day_name(locale='es_ES')
dimDate['quarter'] = dimDate['date'].dt.quarter
dimDate['is_weekend'] = dimDate['date'].dt.weekday >= 5
dimDate['date'] = pd.to_datetime(raw_orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [279]:
dimTime = pd.to_datetime(raw_orders['time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H%M').astype(int).to_frame(name='time_id')
dimTime = dimTime.drop_duplicates()
dimTime['time'] = pd.to_datetime(raw_orders['time'], format='%H:%M:%S', errors='coerce')
dimTime['hour'] = dimTime['time'].dt.hour
dimTime['minutes'] = dimTime['time'].dt.minute
dimTime['time_ampm'] = dimTime['time'].dt.strftime('%I:%M %p')

time_conditions=[
    (dimTime['hour'] >= 0) & (dimTime['hour'] < 6),
    (dimTime['hour'] >= 6) & (dimTime['hour'] < 12),
    (dimTime['hour'] >= 12) & (dimTime['hour'] < 19),
    (dimTime['hour'] >= 19)
]

time_results = ['night','morning', 'afternoon', 'night']

dimTime['moment_day'] = np.select(time_conditions, time_results, default='Unknown')
dimTime['is_rushhour'] = (((dimTime['hour'] >= 6) & (dimTime['hour'] < 9)) | ((dimTime['hour'] >= 18) & (dimTime['hour'] < 20)))
dimTime['time'] = pd.to_datetime(raw_orders['time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')


In [ ]:
dimPizza = pd.merge(raw_pizzas, raw_pizza_types, how= "left", on = "pizza_type_id")
dimPizza['id'] = range(1, len(dimPizza) + 1)
dimPizza = dimPizza.drop(
    columns = ['pizza_type_id', 'ingredients']
)

In [318]:
factSales = pd.merge(raw_orders, raw_orders_details, how='left', on='order_id')

In [319]:
factSales.info()

<class 'pandas.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   order_id          48620 non-null  int64
 1   date              48620 non-null  str  
 2   time              48620 non-null  str  
 3   order_details_id  48620 non-null  int64
 4   pizza_id          48620 non-null  str  
 5   quantity          48620 non-null  int64
dtypes: int64(3), str(3)
memory usage: 3.6 MB


In [ ]:
factSales = pd.merge(factSales, dimDate[['date_id','date']], left_on='date', right_on='date', how="left")
factSales['time_id'] = pd.to_datetime(factSales['time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H%M').astype(int)
factSales = pd.merge(factSales, dimTime[['time_id']], on='time_id', how="left")
factSales = pd.merge(factSales, dimPizza[['id','pizza_id','price']], on = 'pizza_id', how='left')

In [326]:
factSales['total_price'] = factSales['quantity'] * factSales['price']

In [328]:
factSales = factSales.drop(
    columns = ['date','time','pizza_id']
)
factSales = factSales.rename(
    columns = {
        'price': 'unit_price',
        'id': 'pizza_id'
    }
)
factSales = factSales[['order_id','order_details_id','date_id','time_id','pizza_id','quantity','unit_price','total_price']]